# Presentation
This repository contains a fully automated Python pipeline that scrapes league tables, recent results, and upcoming fixtures from **SoccerStats.com** for 60+ leagues from different continents and divisions (e.g., D1, D2, and D3 where applicable). The script also processes the data, and computes a set of probability indicators for matches finishing with **over 2.5 goals**.

The script extracts historical results, upcoming fixtures, team scoring profiles, and builds combined probability estimates based on home/away tendencies and statistical variance.  

The output is delivered as Excel workbooks (full data and treated data) designed for quick filtering, ranking, and decision support.

---

## What this project does
- Scrapes performance tables (total number of games, last 8, home, away) and parses results/fixtures per league.  
- Builds team-level profiles of goals scored and conceded based on:
  - all fixtures played so far, last 8 matches, home games, and away games.  
  - mean and standard deviation (goals scored (GS)/goals conceded (GC)).
- Derives match probabilities for games finishing with over 2.5 goals using:
  - low and average bounds based on the previous number of games that finished over 2.5 goals.  
  - home/away over 2.5 goals tendencies.  
  - GS/GC means and SDs combined (worst- and best-case expectations).  
- Ranks upcoming matches using a multi-criteria sort (low-bound prob, average home/away prob, sums of GS/GC means & SDs, games played).  
- Exports tidy, analysis-ready Excel files (full + treated).

---

## What is produced
The pipeline creates two Excel workbooks:

### 1️⃣ Full Data
**Filename:** `FullDatabase+2.5Goals_<DD-MM-YYYY>.xlsx`  
**Sheets:**
- `OverUnderGoalsTotalFullTime`:
  - league information
  - games played (GP)
  - information about next match
  - low and high bound probability of the games finishing with +2.5 goals (based on the proportion of previous home and away matches that each team’s games ended with over 2.5 goals)
  - probability (home and away) based on the proportion of previous matches in which the home team’s home games and the away team’s away games ended with over 2.5 goals
  - goals scored and conceded by the home team at home games (means and SDs)
  - goals scored and conceded by the away team at away games (means and SDs)  
- `Results` – parsed historical results (date, teams, score)  
- `Fixtures` – upcoming fixtures (date, time, teams)

### 2️⃣ Treated Data
**Filename:** `Treated_+2.5Goals_<DD-MM-YYYY>.xlsx`  
**Sheet:**
- `TreatedData` – filtered view with:
  - GP ≥ 12  
  - low-bound home/away O2.5 ≥ 50.    
  - Defines the best and worst case scenarios for a match finishing with over 2.5 goals:
    - *Worst-case scenario*:
      - Home team’s expected goals = the lower value between their average goals scored at home and the average goals conceded by the away team.
      - Away team’s expected goals = the lower value between their average goals scored away and the average goals conceded by the home team.
    - *Best-case scenario*:
      - Home team’s expected goals = the higher value between their average goals scored at home and the average goals conceded by the away team.
      - Away team’s expected goals = the higher value between their average goals scored away and the average goals conceded by the home team.

---

## How the script works
### 🔹 Scraping & parsing
- Fetches league pages from **SoccerStats.com**, parses the main table (`id="btable"`), and keeps O/U rows.  
- From the results/fixtures pages, identifies rows by a date pattern (e.g., `Mon 4 Nov`), and splits results vs fixtures.  

### 🔹 Team metrics
- For each team, the script collects GS/GC series for all fixtures played so far, last 8 matches, home games, and away games.  
- Compute mean and SD per series.  

### 🔹 Match enrichment
- Create “Next match” strings and propagate per-team metrics.  
- Compute low and high bound probabilities of games finishing over 2.5 goals.  
- Combine GS and GC means & SDs.  

### 🔹 Ranking & export
- Estimates the best and worst case scenarios and ranks teams according to the highest probability of their matches finishing with over 2.5 goals.    
- Write Full Data and Treated Data workbooks.

# 0 - Imports librarys

In [ ]:
import requests, warnings
import pandas as pd
from bs4 import BeautifulSoup
import re, os
import statistics
from datetime import datetime, timedelta
from IPython.display import HTML
from io import BytesIO

In [ ]:
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# 1. "Full Data" database

# 1.1. Web Scraping and Pre-Processing of +2.5 Goals per Game Statistics from 62 Football Leagues  
This section of code retrieves, parses, and structures match performance data from [SoccerStats.com](https://www.soccerstats.com).  

It performs:  
- **Data retrieval:** Downloads a metadata Excel file (`URLs.xlsx`) containing league names, continents, and scraping endpoints.  
- **Automated scraping:** Iterates through 62 league pages to extract tables with team goal statistics (total, last 8 matches, home, and away performance).  
- **Data parsing:** Cleans raw HTML tables, filters out non-relevant rows (e.g., league averages), and organizes the data by team and match context.  
- **Data structuring:** Compiles clean datasets into four Pandas DataFrames (`Total`, `Last8`, `Home`, `Away`) with unified headers and standardized formats.  
- **Data cleaning:** Removes percentage symbols, converts numeric fields, and sorts teams by the proportion of matches with **over 2.5 goals**.  
- **Preview generation:** Displays the cleaned and ranked league tables in scrollable HTML format for easy inspection.

## 1.1.1. Importing League and Endpoint Configuration from GitHub

In [ ]:
# URL of the Excel file containing league and endpoint metadata
url = "https://raw.githubusercontent.com/FabioATMonteiro92/WebScrappingFootballProbabilityOver2.5GoalsPerGame/main/URLs.xlsx"

# Download the Excel file from GitHub
response = requests.get(url)

# Raise an error if the request fails
response.raise_for_status()  # ensure the request succeeded

# Load the Excel content into a pandas DataFrame
dfURLs = pd.read_excel(BytesIO(response.content))

# Configure pandas to display all columns and up to 10 rows
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 10)

# Display the DataFrame preview
dfURLs

## 1.1.2. Web Scraping League Performance Tables from SoccerStats.com

In [ ]:
# Extract relevant URL and league info lists from the URLs DataFrame
ListUrls   = dfURLs["TotalMatchGoalStats"].tolist()
Continent  = dfURLs["Continent"].tolist()
League     = dfURLs["League"].tolist()

# Initialize empty lists to store scraped data for different views
rowsTotalFullTime = []
rowsLast8FullTime = []
rowsHomeFullTime = []
rowsAwayFullTime = []

# Loop through all league URLs to scrape data
IndexContLeague = 0
for i in ListUrls:
    # Send GET request to SoccerStats league page
    response = requests.get(i)
    # Print URL if request fails
    if response.status_code != 200:
        print(i)

    # Parse HTML content of the page
    soup = BeautifulSoup(response.content, 'html.parser')

    # Locate main stats table by ID
    tables = soup.find_all('table', {'id': 'btable'})

    # Extract all rows from the tables
    rows = []
    # Loop through all tables and their rows
    for table in tables:
        for tr in table.find_all('tr')[1:]:  # Skip header
            cells = tr.find_all('td')
            row = [cell.text.strip() for cell in cells]
            rows.append(row)
    # Keep only rows containing percentage values
    rows = [sublist for sublist in rows if any('%' in item for item in sublist)]
    # Remove “League average” summary rows
    rows = [sublist for sublist in rows if 'League average' not in sublist]

    # Dictionary to count team appearances (to separate total, last8, home, away)
    team_counts = {}

    # Categorize rows into Total, Last8, Home, and Away lists
    for sublist in rows:
        team = sublist[0]

        # Initialize team count if not already present
        if team not in team_counts:
            team_counts[team] = 0

        # Increment team appearance count
        team_counts[team] += 1

        # Append row to corresponding category based on team occurrence order
        if team_counts[team] == 1:
            sublist.insert(0,Continent[IndexContLeague])
            sublist.insert(1, League[IndexContLeague])
            rowsTotalFullTime.append(sublist)
        elif team_counts[team] == 2:
            sublist.insert(0,Continent[IndexContLeague])
            sublist.insert(1, League[IndexContLeague])
            rowsLast8FullTime.append(sublist)
        elif team_counts[team] == 3:
            sublist.insert(0,Continent[IndexContLeague])
            sublist.insert(1, League[IndexContLeague])
            rowsHomeFullTime.append(sublist)
        elif team_counts[team] == 4:
            sublist.insert(0,Continent[IndexContLeague])
            sublist.insert(1, League[IndexContLeague])
            rowsAwayFullTime.append(sublist)
    # Move to next league
    IndexContLeague += 1

# Define column headers for all generated DataFrames
headers = ["Continent","League","Team","GP","Avg","0.5+","1.5+","2.5+","3.5+","4.5+","5.5+","BTS","CS","FTS","WTN","LTN"]

# Convert all collected lists into DataFrames
dfOverUnderGoalsTotalFullTime = pd.DataFrame(rowsTotalFullTime, columns=headers)
dfOverUnderGoalsLast8FullTime = pd.DataFrame(rowsLast8FullTime, columns=headers)
dfOverUnderGoalsHomeFullTime = pd.DataFrame(rowsHomeFullTime, columns=headers)
dfOverUnderGoalsAwayFullTime = pd.DataFrame(rowsAwayFullTime, columns=headers)

### 1.1.3. Converting Percentage Values and Sorting by Over 2.5 Goals

In [ ]:
# Define the columns containing percentage values to clean and convert
columns_to_convert = ['0.5+', '1.5+', '2.5+', '3.5+', '4.5+', '5.5+', 'BTS', 'CS', 'FTS', 'WTN', 'LTN']

# Loop through each percentage column and convert from string (with %) to integer
for column in columns_to_convert:
    dfOverUnderGoalsTotalFullTime[column] = dfOverUnderGoalsTotalFullTime[column].str.replace('%', '').astype(int)
    dfOverUnderGoalsLast8FullTime[column] = dfOverUnderGoalsLast8FullTime[column].str.replace('%', '').astype(int)
    dfOverUnderGoalsHomeFullTime[column] = dfOverUnderGoalsHomeFullTime[column].str.replace('%', '').astype(int)
    dfOverUnderGoalsAwayFullTime[column] = dfOverUnderGoalsAwayFullTime[column].str.replace('%', '').astype(int)

# Sort each DataFrame by the percentage of +2.5 goals (and games played) in descending order
dfOverUnderGoalsTotalFullTime = dfOverUnderGoalsTotalFullTime.sort_values(by=['2.5+','GP'], ascending=[False,False])
dfOverUnderGoalsTotalFullTime.reset_index(drop=True,inplace=True)
dfOverUnderGoalsLast8FullTime = dfOverUnderGoalsLast8FullTime.sort_values(by=['2.5+','GP'], ascending=[False,False])
dfOverUnderGoalsLast8FullTime.reset_index(drop=True,inplace=True)
dfOverUnderGoalsHomeFullTime = dfOverUnderGoalsHomeFullTime.sort_values(by=['2.5+','GP'], ascending=[False,False])
dfOverUnderGoalsHomeFullTime.reset_index(drop=True,inplace=True)
dfOverUnderGoalsAwayFullTime = dfOverUnderGoalsAwayFullTime.sort_values(by=['2.5+','GP'], ascending=[False,False])
dfOverUnderGoalsAwayFullTime.reset_index(drop=True,inplace=True)

In [ ]:
# Display the DataFrame as an HTML table with striped styling
HTML(dfOverUnderGoalsTotalFullTime.to_html(classes="table-striped"))

# Display the same DataFrame with scrollable HTML styling for large tables
HTML(
    dfOverUnderGoalsTotalFullTime.to_html()
    .replace('<table border="1" class="dataframe">', 
             '<table border="1" class="dataframe" style="display:block; height:300px; overflow-y:scroll;">')
)

## 1.2. Web Scraping and Pre-Processing of Information Regarding Previous Results and Fixtures from 62 Football Leagues 
This section extends the scraping pipeline to retrieve **historical match results** and **upcoming fixtures** from the *Results by Date* pages on [SoccerStats.com](https://www.soccerstats.com).  

It performs:  
- **URL extraction:** Loads results-by-date endpoints for each league from the metadata file (`URLs.xlsx`).  
- **Automated scraping:** Iterates through every league’s results page to collect both completed matches and scheduled fixtures.  
- **Data parsing:**  
  - Identifies valid table rows containing date-formatted entries (e.g., “Mon 4 Nov”).  
  - Separates past results (rows with scores `"x - y"`) from upcoming fixtures (rows with kickoff times `"hh:mm"`).  
- **Data cleaning:**  
  - Removes irrelevant rows and trims each record to include only key match fields (date, home team, score/time, away team).  
  - Splits combined score strings into distinct numeric columns for home and away goals.  
- **Data structuring:**  
  - Builds two organized Pandas DataFrames:  
    - `dfresults` → historical match results (with goals scored).  
    - `dffixtures` → upcoming fixtures (with match schedules).  
  - Creates a **unique list of team names** for downstream statistical aggregation and modeling.

In [ ]:
# Extract list of results-by-date URLs from the URLs table
ListUrlsbyDate = dfURLs["ResultsByDate"].tolist()

# Initialize containers for parsed past results and upcoming fixtures
rows_results = []
rows_fixtures = []
# Iterate over each league index/URL pair to scrape data
IndexContLeague = 0
for i in ListUrlsbyDate:
    # Send HTTP GET request to the results-by-date page
    response = requests.get(i)
    # Log the URL if the request failed
    if response.status_code != 200:
        print(i)

    # Parse the page HTML with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all main data tables by their id
    tables = soup.find_all('table', {'id': 'btable'})

    # Collect all table rows’ text into a flat list
    rows = []
    # Iterate tables and strip header rows to extract cell text
    for table in tables:
        for tr in table.find_all('tr')[1:]:  # Skip the header row
            cells = tr.find_all('td')
            row = [cell.text.strip() for cell in cells]
            rows.append(row)

    # Compile regex matching 'Mon 4 Nov' style date strings
    pattern = re.compile(r'^[A-Za-z]{3} [0-9]{1,2} [A-Za-z]{3}$')

    # Keep only rows whose first element matches the date pattern
    rows = [sublist for sublist in rows if pattern.match(sublist[0])]
    # Trim each row to the first four fields (date, home, score/time, away)
    for i in range(0,len(rows)):
        rows[i] = rows[i][:4]

    # Split rows into past results (with '-') vs fixtures (with ':') and tag with league/continent
    for i in rows:
        if "-" in i[2]:
            i.insert(0,Continent[IndexContLeague])
            i.insert(1, League[IndexContLeague])
            rows_results.append(i)
        elif ":" in i[2]:
          i.insert(0, Continent[IndexContLeague])
          i.insert(1, League[IndexContLeague])
          rows_fixtures.append(i)
    # Advance to the next league’s indices
    IndexContLeague += 1

# Normalize result score strings into separate numeric home/away goal columns
for i in range(0,len(rows_results)):
    score = rows_results[i][4].split(' - ')
    temp = [rows_results[i][0], rows_results[i][1], rows_results[i][2], rows_results[i][3], int(score[0]), '-', int(score[1]), rows_results[i][5]]
    rows_results[i] = temp

# Define column headers for results and fixtures DataFrames
headers_results = ["Continent","League","Date","Home team","Goals Home","Hifen","Goals Away","Away Team"]
headers_fixtures = ["Continent","League","Date","Home team","Schedule","Away Team"]

# Build DataFrames for historical results and upcoming fixtures
dfresults = pd.DataFrame(rows_results, columns=headers_results)
dffixtures = pd.DataFrame(rows_fixtures, columns=headers_fixtures)

# Create a de-duplicated list of team names (using home-team entries as canonical)
listTeamNamesUnique =  dfresults["Home team"].drop_duplicates().tolist()

## 1.3. Calculating Average and Standard Deviation of Goals Scored and Conceded
This section transforms raw match results into **team-level statistical summaries** that describe scoring and defensive performance across multiple contexts.  

It produces a series of dictionaries capturing **average goals scored (GS)** and **goals conceded (GC)** per team, along with their **standard deviations (SD)**, both overall and under specific conditions (last 8 matches, home, and away).  

### 1.3.1. All Fixtures Played so Far

In [ ]:
# Initialize dict of goals scored per team across all matches
##########################################################################################################################################################################
# Dict 1.2.1a: build GoalsbyTeamTotalDict (all fixtures) for dfOverUnderGoalsTotalFullTime
GoalsbyTeamTotalDict = {team: [] for team in listTeamNamesUnique}

# Iterate all results to collect goals scored by each team (home and away)
for index, row in dfresults.iterrows():
    # Append home goals to the home team's list
    home_team = row['Home team']
    home_goals = row['Goals Home']
    if home_team in GoalsbyTeamTotalDict:
        GoalsbyTeamTotalDict[home_team].append(home_goals)

    # Append away goals to the away team's list
    away_team = row['Away Team']
    away_goals = row['Goals Away']
    if away_team in GoalsbyTeamTotalDict:
        GoalsbyTeamTotalDict[away_team].append(away_goals)

# Copy scored-goals dict to compute mean and stdev per team
AverageSDGoalsbyTeamTotalDict = GoalsbyTeamTotalDict.copy()

# Replace each team’s list with [mean, stdev] (or value and '-' if only one game)
for team, goals in AverageSDGoalsbyTeamTotalDict.items():
    if len(goals) == 1:
        AverageSDGoalsbyTeamTotalDict[team] = [goals[0], '-']
    else:
        average = round(statistics.mean(goals), 2)
        stdev = round(statistics.stdev(goals), 2)
        AverageSDGoalsbyTeamTotalDict[team] = [average, stdev]
        
# Initialize dict of goals conceded per team across all matches
#################################################################
# Dict 1.2.1b: build GoalsConcededbyTeamTotalDict (all fixtures) for dfOverUnderGoalsTotalFullTime
# Create a new dictionary to store goals conceded by each team
GoalsConcededbyTeamTotalDict = {team: [] for team in listTeamNamesUnique}

# Iterate all results to collect goals conceded by each team
for index, row in dfresults.iterrows():
    # Home team concedes the away team's goals
    home_team = row['Home team']
    away_goals = row['Goals Away']
    if home_team in GoalsConcededbyTeamTotalDict:
        GoalsConcededbyTeamTotalDict[home_team].append(away_goals)

    # Away team concedes the home team's goals
    away_team = row['Away Team']
    home_goals = row['Goals Home']
    if away_team in GoalsConcededbyTeamTotalDict:
        GoalsConcededbyTeamTotalDict[away_team].append(home_goals)

# Compute [mean, stdev] for goals conceded per team (or value and '-' if only one game)
AverageSDGoalsConcededbyTeamTotalDict = {}
for team, goals_conceded in GoalsConcededbyTeamTotalDict.items():
    if len(goals_conceded) == 1:  # Handle single observation case
        AverageSDGoalsConcededbyTeamTotalDict[team] = [goals_conceded[0], '-']
    else:
        average_gc = round(statistics.mean(goals_conceded), 2)
        stdev_gc = round(statistics.stdev(goals_conceded), 2)
        AverageSDGoalsConcededbyTeamTotalDict[team] = [average_gc, stdev_gc]

# Print first 10 entries from the scored-goals averages dict for a quick sanity check
for i, (key, value) in enumerate(AverageSDGoalsbyTeamTotalDict.items()):
    if i == 10:
        break
    print(key, ":", value)

### 1.3.2. Last 8 Games

In [ ]:
# Build dict of last-8 goals scored per team using the total dict as a base
#Dict 1.2.2a: GoalsbyTeamLast8Dict for dfOverUnderGoalsLast8FullTime
GoalsbyTeamLast8Dict = GoalsbyTeamTotalDict.copy()

# For each team, keep last 8 goals if available; otherwise mark existing entries with '-'
for team in GoalsbyTeamLast8Dict:
    if len(GoalsbyTeamLast8Dict[team]) < 8:
        GoalsbyTeamLast8Dict[team] = ['-'] * len(GoalsbyTeamLast8Dict[team])
    else:
        GoalsbyTeamLast8Dict[team] = GoalsbyTeamLast8Dict[team][-8:]

# Copy to compute mean and stdev for last-8 goals scored
AverageSDGoalsbyTeamLast8Dict = GoalsbyTeamLast8Dict.copy()

# Replace lists with [mean, stdev]; handle single-entry and '-' placeholders
for team, goals in AverageSDGoalsbyTeamLast8Dict.items():
    if len(goals) == 1:
        AverageSDGoalsbyTeamLast8Dict[team] = [goals[0], '-']
    elif "-" in goals:
        AverageSDGoalsbyTeamLast8Dict[team] = ['-', '-']
    else:
        average = round(statistics.mean(goals), 2)
        stdev = round(statistics.stdev(goals), 2)
        AverageSDGoalsbyTeamLast8Dict[team] = [average, stdev]

# Build dict of last-8 goals conceded per team using total conceded dict as a base
#################################################################
#Dict 1.2.2b: GoalsConcededbyTeamLast8Dict for dfOverUnderGoalsLast8FullTime
# Create a new dictionary to store goals conceded by each team
GoalsConcededbyTeamLast8Dict = GoalsConcededbyTeamTotalDict.copy()

# For each team, keep last 8 conceded goals if available; otherwise mark with '-'
for team in GoalsConcededbyTeamLast8Dict:
    if len(GoalsConcededbyTeamLast8Dict[team]) < 8:
        GoalsConcededbyTeamLast8Dict[team] = ['-'] * len(GoalsConcededbyTeamLast8Dict[team])
    else:
        GoalsConcededbyTeamLast8Dict[team] = GoalsConcededbyTeamLast8Dict[team][-8:]

# Compute [mean, stdev] for last-8 conceded; handle single-entry and '-' placeholders
AverageSDGoalsConcededbyTeamLast8Dict = {}
for team, goals_conceded in GoalsConcededbyTeamLast8Dict.items():
    if len(goals_conceded) == 1:
        AverageSDGoalsConcededbyTeamLast8Dict[team] = [goals_conceded[0], '-']
    elif "-" in goals_conceded:
        AverageSDGoalsConcededbyTeamLast8Dict[team] = ['-', '-']
    else:
        average_gc = round(statistics.mean(goals_conceded), 2)
        stdev_gc = round(statistics.stdev(goals_conceded), 2)
        AverageSDGoalsConcededbyTeamLast8Dict[team] = [average_gc, stdev_gc]
        
# Print the first 10 entries for a quick check of the conceded dict
for i, (key, value) in enumerate(AverageSDGoalsConcededbyTeamLast8Dict.items()):
    if i == 10:
        break
    print(key, ":", value)

### 1.3.3. Home Games

In [ ]:
# Build dictionary of goals scored at home by each team
#Dict 1.2.3a: GoalsbyTeamHomeDict for dfOverUnderGoalsHomeFullTime
GoalsbyTeamHomeDict = {team: [] for team in listTeamNamesUnique}

# Iterate through match results and record goals scored by home teams
for index, row in dfresults.iterrows():
    # Extract and store home team goals
    home_team = row['Home team']
    home_goals = row['Goals Home']
    if home_team in GoalsbyTeamHomeDict:
        GoalsbyTeamHomeDict[home_team].append(home_goals)

# Copy home-goals dictionary to calculate average and standard deviation
AverageSDGoalsbyTeamHomeDict = GoalsbyTeamHomeDict.copy()

# Replace each team's goal list with [mean, stdev]; use '-' when only one match exists
for team, goals in AverageSDGoalsbyTeamHomeDict.items():
    if len(goals) == 1:
        AverageSDGoalsbyTeamHomeDict[team] = [goals[0], '-']
    else:
        average = round(statistics.mean(goals), 2)
        stdev = round(statistics.stdev(goals), 2)
        AverageSDGoalsbyTeamHomeDict[team] = [average, stdev]

# Build dictionary of goals conceded at home by each team
#################################################################
#Dict 1.2.3b: GoalsConcededbyTeamHomeDict for dfOverUnderGoalsHomeFullTime
GoalsConcededbyTeamHomeDict = {team: [] for team in listTeamNamesUnique}

# Iterate through results to record goals conceded by home teams
for index, row in dfresults.iterrows():
    # Home team concedes the away team's goals
    home_team = row['Home team']
    away_goals = row['Goals Away']
    if home_team in GoalsConcededbyTeamHomeDict:
        GoalsConcededbyTeamHomeDict[home_team].append(away_goals)

# Calculate average and standard deviation for goals conceded at home
AverageSDGoalsConcededbyTeamHomeDict = {}
for team, goals_conceded in GoalsConcededbyTeamHomeDict.items():
    if len(goals_conceded) == 1:
        AverageSDGoalsConcededbyTeamHomeDict[team] = [goals_conceded[0], '-']
    else:
        average_gc = round(statistics.mean(goals_conceded), 2)
        stdev_gc = round(statistics.stdev(goals_conceded), 2)
        AverageSDGoalsConcededbyTeamHomeDict[team] = [average_gc, stdev_gc]

# Display first 10 entries from the home conceded goals dictionary for verification
for i, (key, value) in enumerate(AverageSDGoalsConcededbyTeamHomeDict.items()):
    if i == 10:
        break
    print(key, ":", value)

### 1.3.4. Away Games

In [ ]:
# Build dict of goals scored away from home for each team
#Dict 1.2.4a: GoalsbyTeamAwaylDict for dfOverUnderGoalsAwayFullTime
GoalsbyTeamAwaylDict = {team: [] for team in listTeamNamesUnique}

# Iterate over match results and collect goals scored by away teams
for index, row in dfresults.iterrows():
    # Append away goals to corresponding team's record
    away_team = row['Away Team']
    away_goals = row['Goals Away']
    if away_team in GoalsbyTeamAwaylDict:
        GoalsbyTeamAwaylDict[away_team].append(away_goals)

# Copy dict to compute mean and standard deviation of away goals per team
AverageSDGoalsbyTeamAwaylDict = GoalsbyTeamAwaylDict.copy()

# Replace lists with [mean, stdev] (or value and '-' if team has only one away match)
for team, goals in AverageSDGoalsbyTeamAwaylDict.items():
    if len(goals) == 1:
        AverageSDGoalsbyTeamAwaylDict[team] = [goals[0], '-']
    else:
        average = round(statistics.mean(goals), 2)
        stdev = round(statistics.stdev(goals), 2)
        AverageSDGoalsbyTeamAwaylDict[team] = [average, stdev]

# Build dict of goals conceded away (away teams conceding home teams' goals)
#################################################################
#Dict 1.2.4b: GoalsbyTeamAwayDict for dfOverUnderGoalsTotalFullTime
# Create a new dictionary to store goals conceded by each away team
GoalsConcededbyTeamAwayDict = {team: [] for team in listTeamNamesUnique}

# Iterate through results and record goals conceded by away teams
for index, row in dfresults.iterrows():
    # Away team concedes home team's goals; append to record
    away_team = row['Away Team']
    home_goals = row['Goals Home']
    if away_team in GoalsConcededbyTeamAwayDict:
        GoalsConcededbyTeamAwayDict[away_team].append(home_goals)

# Compute [mean, stdev] for goals conceded away (or value and '-' if single match)
AverageSDGoalsConcededbyTeamAwayDict = {}
for team, goals_conceded in GoalsConcededbyTeamAwayDict.items():
    if len(goals_conceded) == 1:
        AverageSDGoalsConcededbyTeamAwayDict[team] = [goals_conceded[0], '-']  # Handle single observation
    else:
        average_gc = round(statistics.mean(goals_conceded), 2)
        stdev_gc = round(statistics.stdev(goals_conceded), 2)
        AverageSDGoalsConcededbyTeamAwayDict[team] = [average_gc, stdev_gc]

# Print first 10 entries from away conceded dict as a validation sample
for i, (key, value) in enumerate(AverageSDGoalsConcededbyTeamAwayDict.items()):
    if i == 10:
        break
    print(key, ":", value)

### 1.3.5. Adds the dicitonary with the averages and SD of to the dataframes created in section 1

In [ ]:
# Map per-team overall goals-scored mean and stdev into the Total table
#########################################################################################################################################################################
#Adds the dicitonary with the averages and SD of the teams to the dataframes with over ubnder goals information sorted by percentage of games with 2.5- goals
dfOverUnderGoalsTotalFullTime['average_GS'] = dfOverUnderGoalsTotalFullTime['Team'].map(lambda x: AverageSDGoalsbyTeamTotalDict[x][0])
# Map per-team overall GS stdev into the Total table
dfOverUnderGoalsTotalFullTime['SD_GS'] = dfOverUnderGoalsTotalFullTime['Team'].map(lambda x: AverageSDGoalsbyTeamTotalDict[x][1])

# Map per-team last-8 goals-scored mean into the Last8 table
dfOverUnderGoalsLast8FullTime['average_GS'] = dfOverUnderGoalsLast8FullTime['Team'].map(lambda x: AverageSDGoalsbyTeamLast8Dict[x][0])
# Map per-team last-8 goals-scored stdev into the Last8 table
dfOverUnderGoalsLast8FullTime['SD_GS'] = dfOverUnderGoalsLast8FullTime['Team'].map(lambda x: AverageSDGoalsbyTeamLast8Dict[x][1])

# Map per-team home goals-scored mean into the Home table
dfOverUnderGoalsHomeFullTime['average_GS'] = dfOverUnderGoalsHomeFullTime['Team'].map(lambda x: AverageSDGoalsbyTeamHomeDict[x][0])
# Map per-team home goals-scored stdev into the Home table
dfOverUnderGoalsHomeFullTime['SD_GS'] = dfOverUnderGoalsHomeFullTime['Team'].map(lambda x: AverageSDGoalsbyTeamHomeDict[x][1])

# Map per-team away goals-scored mean into the Away table
dfOverUnderGoalsAwayFullTime['average_GS'] = dfOverUnderGoalsAwayFullTime['Team'].map(lambda x: AverageSDGoalsbyTeamAwaylDict[x][0])
# Map per-team away goals-scored stdev into the Away table
dfOverUnderGoalsAwayFullTime['SD_GS'] = dfOverUnderGoalsAwayFullTime['Team'].map(lambda x: AverageSDGoalsbyTeamAwaylDict[x][1])

# Map per-team overall goals-conceded mean into the Total table (with safe default)
# Add the averages and standard deviations for goals conceded to the dataframe
dfOverUnderGoalsTotalFullTime['average_GC'] = dfOverUnderGoalsTotalFullTime['Team'].map(lambda x: AverageSDGoalsConcededbyTeamTotalDict.get(x, [None, None])[0])
# Map per-team overall goals-conceded stdev into the Total table (with safe default)
dfOverUnderGoalsTotalFullTime['SD_GC'] = dfOverUnderGoalsTotalFullTime['Team'].map(lambda x: AverageSDGoalsConcededbyTeamTotalDict.get(x, [None, None])[1])

# Map per-team last-8 goals-conceded mean into the Last8 table (with safe default)
dfOverUnderGoalsLast8FullTime['average_GC'] = dfOverUnderGoalsLast8FullTime['Team'].map(lambda x: AverageSDGoalsConcededbyTeamLast8Dict.get(x, [None, None])[0])
# Map per-team last-8 goals-conceded stdev into the Last8 table (with safe default)
dfOverUnderGoalsLast8FullTime['SD_GC'] = dfOverUnderGoalsLast8FullTime['Team'].map(lambda x: AverageSDGoalsConcededbyTeamLast8Dict.get(x, [None, None])[1])

# Map per-team home goals-conceded mean into the Home table (with safe default)
dfOverUnderGoalsHomeFullTime['average_GC'] = dfOverUnderGoalsHomeFullTime['Team'].map(lambda x: AverageSDGoalsConcededbyTeamHomeDict.get(x, [None, None])[0])
# Map per-team home goals-conceded stdev into the Home table (with safe default)
dfOverUnderGoalsHomeFullTime['SD_GC'] = dfOverUnderGoalsHomeFullTime['Team'].map(lambda x: AverageSDGoalsConcededbyTeamHomeDict.get(x, [None, None])[1])

# Map per-team away goals-conceded mean into the Away table (with safe default)
dfOverUnderGoalsAwayFullTime['average_GC'] = dfOverUnderGoalsAwayFullTime['Team'].map(lambda x: AverageSDGoalsConcededbyTeamAwayDict.get(x, [None, None])[0])
# Map per-team away goals-conceded stdev into the Away table (with safe default)
dfOverUnderGoalsAwayFullTime['SD_GC'] = dfOverUnderGoalsAwayFullTime['Team'].map(lambda x: AverageSDGoalsConcededbyTeamAwayDict.get(x, [None, None])[1])

# Configure DataFrame display to show all columns and a small number of rows
# Display all columns when printing the DataFrame
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 6)

# Trigger display of the enriched Total table in Jupyter
# Show the cleaned screening dataset
dfOverUnderGoalsTotalFullTime

## 1.4. Calculating Over 2.5 Goals Probabilities and Integrating them into the Main Dataset
This section combines **fixture scheduling data** with **historical goal probability metrics** to enrich the main team performance dataset.  
It identifies each team’s *next upcoming match* (either home or away) and assigns probability estimates for matches likely to exceed 2.5 goals, based on both teams’ past performance.  

---

**This section:**
- Creates a **fixture copy** to ensure data integrity during processing.  
- Extracts **each team’s first upcoming match** (avoiding duplicates across home and away roles).  
- Calculates three probability metrics based on the teams’ historical +2.5 goal rates:  
  - **Low bound probability:** Conservative estimate (lower team value).  
  - **High bound probability:** Optimistic estimate (higher team value).  
  - **Average probability:** Midpoint between low and high bounds.  
- Integrates the above probabilities into the main dataset `dfOverUnderGoalsTotalFullTime`.  
- Maps additional **home and away +2.5% values** from separate home/away DataFrames.  
- Produces an **enriched team-level table** that connects current fixtures to historical over/under performance trends.  

This output lays the groundwork for **predictive modeling** of goal outcomes in upcoming matches across global leagues.

In [ ]:
# Work on a copy of fixtures to avoid mutating the original
dffixtures_forExcel = dffixtures.copy()

# Initialize container for each team's first upcoming appearance (home or away)
first_appearances = pd.DataFrame()

# Track which teams have already been captured as first appearance
home_appeared_teams = set()
away_appeared_teams = set()

# Iterate fixtures to capture the first upcoming row for each team (either as home or away)
for index, row in dffixtures_forExcel.iterrows():
    home_team = row['Home team']
    away_team = row['Away Team']

    # If home team hasn’t appeared yet in either role, record this row and mark it seen
    if home_team not in home_appeared_teams and home_team not in away_appeared_teams:
        first_appearances = pd.concat([first_appearances, pd.DataFrame([row])])
        home_appeared_teams.add(home_team)

    # If away team hasn’t appeared yet in either role, record this row and mark it seen
    if away_team not in home_appeared_teams and away_team not in away_appeared_teams:
        first_appearances = pd.concat([first_appearances, pd.DataFrame([row])])
        away_appeared_teams.add(away_team)

# Reindex and drop league metadata to keep only match-specific fields
first_appearances.reset_index(drop=True, inplace=True)
first_appearances = first_appearances.drop(columns=['Continent', 'League'])

# Keep only team and +2.5% columns for lookups from the Total table
df_relevant = dfOverUnderGoalsTotalFullTime[['Team', '2.5+']]

# Build a fast lookup: team -> +2.5 percentage
team_2_5_plus = df_relevant.set_index('Team')['2.5+'].to_dict()

# Compute a conservative low bound = min(home_team %, away_team %)
first_appearances['low bound probability'] = first_appearances.apply(
    lambda row: min(team_2_5_plus.get(row['Home team'], float('inf')),
                    team_2_5_plus.get(row['Away Team'], float('inf'))), axis=1)

# Compute an optimistic high bound = max(home_team %, away_team %)
first_appearances['high bound probability'] = first_appearances.apply(
    lambda row: max(team_2_5_plus.get(row['Home team'], float('-inf')),
                    team_2_5_plus.get(row['Away Team'], float('-inf'))), axis=1)

# Explicitly preserve computed bound columns (no-op assignment)
first_appearances['low bound probability'] = first_appearances['low bound probability']
first_appearances['high bound probability'] = first_appearances['high bound probability']

# Compute the mid-point probability as the average of low and high bounds
first_appearances['average probability'] = (first_appearances['low bound probability'] + first_appearances['high bound probability']) / 2

# Round the average probability to two decimals for tidy presentation
first_appearances['average probability'] = first_appearances['average probability'].round(2)

# Ensure all target columns exist in the destination table before assignment
required_columns = ['Date', 'Home team', 'Schedule', 'Away Team',
                    'low bound probability', 'high bound probability', 'average probability']

# Add any missing required columns with None defaults
for col in required_columns:
    if col not in dfOverUnderGoalsTotalFullTime.columns:
        dfOverUnderGoalsTotalFullTime[col] = None  # Add the column if it doesn't exist

# Propagate fixture and probability fields into the Total table for both teams involved
for index, row in first_appearances.iterrows():
    home_team = row['Home team']
    away_team = row['Away Team']

    # Locate rows for the home and away teams within the Total table
    matching_home_team = dfOverUnderGoalsTotalFullTime['Team'] == home_team
    matching_away_team = dfOverUnderGoalsTotalFullTime['Team'] == away_team

    # Assign the captured fields to the home team row(s)
    dfOverUnderGoalsTotalFullTime.loc[matching_home_team, ['Date', 'Home team', 'Schedule', 'Away Team',
                                                 'low bound probability', 'high bound probability',
                                                 'average probability']] = row[
        ['Date', 'Home team', 'Schedule', 'Away Team',
         'low bound probability', 'high bound probability', 'average probability']].values

    # Assign the same fields to the away team row(s)
    dfOverUnderGoalsTotalFullTime.loc[matching_away_team, ['Date', 'Home team', 'Schedule', 'Away Team',
                                                 'low bound probability', 'high bound probability',
                                                 'average probability']] = row[
        ['Date', 'Home team', 'Schedule', 'Away Team',
         'low bound probability', 'high bound probability', 'average probability']].values

# Build mappings of team -> home/away +2.5% from the respective tables
home_team_2_5_map = dfOverUnderGoalsHomeFullTime.set_index('Team')['2.5+'].to_dict()
away_team_2_5_map = dfOverUnderGoalsAwayFullTime.set_index('Team')['2.5+'].to_dict()

# Map home/away +2.5% onto the Total table for the specific fixture sides
dfOverUnderGoalsTotalFullTime['probability home team'] = dfOverUnderGoalsTotalFullTime['Home team'].map(home_team_2_5_map)
dfOverUnderGoalsTotalFullTime['probability away team'] = dfOverUnderGoalsTotalFullTime['Away Team'].map(away_team_2_5_map)

# Explicitly preserve mapped probability columns (no-op assignments)
dfOverUnderGoalsTotalFullTime['probability home team'] = dfOverUnderGoalsTotalFullTime['probability home team']
dfOverUnderGoalsTotalFullTime['probability away team'] = dfOverUnderGoalsTotalFullTime['probability away team']

# Configure DataFrame display for full column visibility and a compact row sample
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 6)

# Display the enriched Total table preview
dfOverUnderGoalsTotalFullTime

## 1.5. Creating ‘Next Match’ Summaries and Home/Away Probability Metrics
This section refines and enriches the main dataset by generating a summary of each team’s next fixture and by integrating statistical goal probability metrics derived from prior analysis.  

---

**This section:** 
- Builds a **“Next match”** field combining key fixture details — date, time, and opposing teams — into a single readable string.  
- Removes redundant fixture columns now embedded in the new “Next match” variable.  
- **Reorganizes column order** for improved structure and readability.  
- Calculates two combined goal probability indicators:  
  - **Low bound home/away probability:** Conservative estimate (minimum of home and away +2.5% values).  
  - **Average home/away probability:** Central tendency (mean of both +2.5% values).  
- Rounds probability metrics to two decimals and positions them adjacent for interpretability.  
- Configures the display for concise visualization of the enriched dataset.  

The resulting table provides an organized and interpretable structure linking **fixture details** with **goal-scoring probabilities**, essential for match-level predictive modeling and reporting.

In [ ]:
# Create a backup of the Date column for later use
dfOverUnderGoalsTotalFullTime["Date1"] = dfOverUnderGoalsTotalFullTime["Date"]
# Compose a human-readable “Next match” string from date, time, and teams
dfOverUnderGoalsTotalFullTime['Next match'] = dfOverUnderGoalsTotalFullTime.apply(lambda row: f"{row['Date']}, {row['Schedule']}, {row['Home team']} - {row['Away Team']}", axis=1)

# Remove the original fixture columns now embedded in “Next match”
dfOverUnderGoalsTotalFullTime = dfOverUnderGoalsTotalFullTime.drop(columns=['Date', 'Home team', 'Schedule', 'Away Team'])

# Reorder columns to place “Next match” at position 21 (index 20)
columns = dfOverUnderGoalsTotalFullTime.columns.tolist()  # Get list of columns
columns.insert(20, columns.pop(columns.index('Next match')))  # Move "next match" to position 21 (index 20)
dfOverUnderGoalsTotalFullTime = dfOverUnderGoalsTotalFullTime[columns]  # Reorder the dataframe

# Compute the conservative bound using the smaller of home/away O2.5 percentages
dfOverUnderGoalsTotalFullTime['low bound home/away probability'] = dfOverUnderGoalsTotalFullTime[['probability home team', 'probability away team']].min(axis=1)
# Move the low-bound column to a specific position for readability
columns = dfOverUnderGoalsTotalFullTime.columns.tolist()  # Get list of columns
columns.insert(26, columns.pop(columns.index('low bound home/away probability')))  # Move "next match" to position 21 (index 20)
dfOverUnderGoalsTotalFullTime = dfOverUnderGoalsTotalFullTime[columns]  # Reorder the dataframe

# Compute the average of home/away O2.5 percentages (rounded to 2 decimals)
dfOverUnderGoalsTotalFullTime['average home/away probability'] = round(dfOverUnderGoalsTotalFullTime[['probability home team', 'probability away team']].mean(axis=1),2)
# Move the average column adjacent to the low-bound column
columns = dfOverUnderGoalsTotalFullTime.columns.tolist()  # Get list of columns
columns.insert(27, columns.pop(columns.index('average home/away probability')))  # Move "next match" to position 21 (index 20)
dfOverUnderGoalsTotalFullTime = dfOverUnderGoalsTotalFullTime[columns]  # Reorder the dataframe

# Configure display to show all columns and a compact number of rows
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 6)

# Render the updated table in the notebook
dfOverUnderGoalsTotalFullTime

## 1.6. Integrating Home and Away Goal Metrics into the Dataset
This section enhances the main dataset by linking each upcoming match with detailed team performance metrics derived from historical data. It enriches fixture-level records with averages and standard deviations for goals scored and conceded, both at home and away.  

---

**This section:**  
- Maps team-level performance statistics (previously computed dictionaries) into the fixture table, including:  
  - **Average goals scored at home and away**.  
  - **Standard deviation of goals scored at home and away**.  
  - **Average goals conceded at home and away**.  
  - **Standard deviation of goals conceded at home and away**.  
- Ensures that each fixture now carries **statistical context** about the expected scoring and defensive behavior of both teams. 
- Adjusts display settings to show all columns and a compact number of rows for review.  

In [ ]:
# Helper to parse "Next match" into (home_team, away_team)
# Function to extract home and away team names
def extract_teams(match_info):
    match_details = match_info.split(', ')[-1]  # Get the "Landskrona - Brage" part
    home_team, away_team = match_details.split(' - ')  # Split into home and away teams
    return home_team.strip(), away_team.strip()

# Map home team's avg goals scored at home from dict
# Populate the columns based on the extracted home and away teams
dfOverUnderGoalsTotalFullTime['average_GS_Home'] = dfOverUnderGoalsTotalFullTime['Next match'].map(
    lambda x: AverageSDGoalsbyTeamHomeDict.get(extract_teams(x)[0], [None, None])[0]
)
# Map home team's SD of goals scored at home
dfOverUnderGoalsTotalFullTime['SD_GS_Home'] = dfOverUnderGoalsTotalFullTime['Next match'].map(
    lambda x: AverageSDGoalsbyTeamHomeDict.get(extract_teams(x)[0], [None, None])[1]
)
# Map home team's avg goals conceded at home
dfOverUnderGoalsTotalFullTime['average_GC_Home'] = dfOverUnderGoalsTotalFullTime['Next match'].map(
    lambda x: AverageSDGoalsConcededbyTeamHomeDict.get(extract_teams(x)[0], [None, None])[0]
)
# Map home team's SD of goals conceded at home
dfOverUnderGoalsTotalFullTime['SD_GC_Home'] = dfOverUnderGoalsTotalFullTime['Next match'].map(
    lambda x: AverageSDGoalsConcededbyTeamHomeDict.get(extract_teams(x)[0], [None, None])[1]
)
# Map away team's avg goals scored away
dfOverUnderGoalsTotalFullTime['average_GS_Away'] = dfOverUnderGoalsTotalFullTime['Next match'].map(
    lambda x: AverageSDGoalsbyTeamAwaylDict.get(extract_teams(x)[1], [None, None])[0]
)
# Map away team's SD of goals scored away
dfOverUnderGoalsTotalFullTime['SD_GS_Away'] = dfOverUnderGoalsTotalFullTime['Next match'].map(
    lambda x: AverageSDGoalsbyTeamAwaylDict.get(extract_teams(x)[1], [None, None])[1]
)
# Map away team's avg goals conceded away
dfOverUnderGoalsTotalFullTime['average_GC_Away'] = dfOverUnderGoalsTotalFullTime['Next match'].map(
    lambda x: AverageSDGoalsConcededbyTeamAwayDict.get(extract_teams(x)[1], [None, None])[0]
)
# Map away team's SD of goals conceded away
dfOverUnderGoalsTotalFullTime['SD_GC_Away'] = dfOverUnderGoalsTotalFullTime['Next match'].map(
    lambda x: AverageSDGoalsConcededbyTeamAwayDict.get(extract_teams(x)[1], [None, None])[1]
)

# Temporarily remove Date1 to reposition it
date1_column = dfOverUnderGoalsTotalFullTime.pop('Date1')

# Append Date1 back as the last column
# Add the "Date1" column back at the end of the DataFrame
dfOverUnderGoalsTotalFullTime['Date1'] = date1_column

# Set wide display to see all columns and limit rows
# Display all columns when printing the DataFrame
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 6)

# Render the updated DataFrame
# Show the cleaned screening dataset
dfOverUnderGoalsTotalFullTime

## 1.7. Aggregating Team Scoring and Conceding Tendencies
This section consolidates offensive and defensive performance indicators into comprehensive match-level metrics. By summing the average and standard deviation values for goals scored and conceded by both home and away teams, it produces an overall measure of each fixture’s **expected scoring potential** and **statistical variability**.  

---

**This section:**  
- Calculates **total expected goal activity** by summing average goals scored and conceded for both teams.  
- Computes **aggregate variability** by combining standard deviations from all goal-related metrics.   
- Configures DataFrame display settings for clear inspection of results.  

The resulting table offers a compact yet insightful view of the **combined scoring dynamics** that characterize each upcoming fixture.

In [ ]:
# Compute total expected scoring tendency combining all average GS/GC values
dfOverUnderGoalsTotalFullTime['Sum avg Goals Scored/Conceded'] = dfOverUnderGoalsTotalFullTime[['average_GS_Home', 'average_GC_Home','average_GS_Away', 'average_GC_Away']].sum(axis=1)

# Compute total variability combining all SD GS/GC values
dfOverUnderGoalsTotalFullTime['Sum SD Goals Scored/Conceded'] = dfOverUnderGoalsTotalFullTime[['SD_GS_Home', 'SD_GC_Home','SD_GS_Away', 'SD_GC_Away']].sum(axis=1)

# Configure DataFrame view to show all columns and limited rows
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 6)

# Display the enriched dataset preview
dfOverUnderGoalsTotalFullTime

## 1.8. Sorting and Filtering Matches Based on Over-2.5-Goals Metrics
This section organizes and refines the dataset to highlight the most relevant upcoming fixtures based on their statistical scoring potential - By ranking matches according to conservative and average Over-2.5-goal probabilities, combined scoring metrics, and the number of games played.  

---

**This section:**  
- Ranks all fixtures by:  
  - **Low-bound and average Over-2.5-goal probabilities** (highest first).  
  - **Aggregate goal averages and variability** (to capture scoring tendencies).  
  - **Games played (GP)** to prioritize teams with stronger data reliability.   
- Produces a **scrollable, export-ready HTML preview** of the final ranked match table.  

The resulting dataset provides a **streamlined, insight-focused ranking** of fixtures most likely to feature high goal activity.

In [ ]:
# Rank matches by conservative/average O2.5, then sum of means/SDs, then GP (desc for GP)
dfOverUnderGoalsTotalFullTime = dfOverUnderGoalsTotalFullTime.sort_values(by=['low bound home/away probability','average home/away probability','Sum avg Goals Scored/Conceded','Sum SD Goals Scored/Conceded','GP'], ascending=[False,False,True,True,False])

# Drop the last two columns (cleanup before exporting)
dfOverUnderGoalsTotalFullTime = dfOverUnderGoalsTotalFullTime.iloc[:, :-2]

# Reset row index after sorting/trimming
dfOverUnderGoalsTotalFullTime.reset_index(drop=True,inplace=True)

# Sort Last-8 table by +2.5% and GP (both descending)
dfOverUnderGoalsLast8FullTime = dfOverUnderGoalsLast8FullTime.sort_values(by=['2.5+','GP'], ascending=[False,False])
# Reset index for Last-8 table
dfOverUnderGoalsLast8FullTime.reset_index(drop=True,inplace=True)

# Sort Home table by +2.5% and GP (both descending)
dfOverUnderGoalsHomeFullTime = dfOverUnderGoalsHomeFullTime.sort_values(by=['2.5+','GP'], ascending=[False,False])
# Reset index for Home table
dfOverUnderGoalsHomeFullTime.reset_index(drop=True,inplace=True)

# Sort Away table by +2.5% and GP (both descending)
dfOverUnderGoalsAwayFullTime = dfOverUnderGoalsAwayFullTime.sort_values(by=['2.5+','GP'], ascending=[False,False])
# Reset index for Away table
dfOverUnderGoalsAwayFullTime.reset_index(drop=True,inplace=True)

# Remove duplicate fixtures, keeping the first-ranked occurrence per “Next match”
dfOverUnderGoalsTotalFullTime_v2_0 = dfOverUnderGoalsTotalFullTime.drop_duplicates(subset='Next match', keep='first')

# Drop raw percentage and intermediate stat columns for a lean export view
dfOverUnderGoalsTotalFullTime_v2_0 = dfOverUnderGoalsTotalFullTime_v2_0.drop(columns=[f'Team','Avg', '0.5+', '1.5+', '2.5+', '3.5+', '4.5+', '5.5+', 'BTS', 'CS', 'FTS', 'WTN', 'LTN', 'average_GS', 'SD_GS', 'average_GC','SD_GC'])

# Reset index after deduplication/column drops
dfOverUnderGoalsTotalFullTime_v2_0.reset_index(drop=True, inplace=True)

# Render the final ranked table (striped style) for preview
HTML(dfOverUnderGoalsTotalFullTime_v2_0.to_html(classes="table-striped"))

# Render a scrollable HTML view suitable for large tables
HTML(
    dfOverUnderGoalsTotalFullTime_v2_0.to_html()
    .replace('<table border="1" class="dataframe">', 
             '<table border="1" class="dataframe" style="display:block; height:300px; overflow-y:scroll;">')
)

## 1.9. Downloading the "Full Data" dataset
This section provides access to the cleaned and processed version of the "Full Data" dataset.

If you wish to download this dataset, simply run this cell.  

**Note:** Binder does *not* automatically download files.  

To export the dataset:

1. Click the **folder icon** on the left-hand sidebar of the Binder interface.  
2. Locate the generated `.xlsx` file named "FullDatabase+2.5Goals_DD-MM-YYYY.xlsx".  
3. Right-click the file and select **“Download.”**

In [ ]:
# Import FileLink for optional download links in Jupyter
from IPython.display import FileLink

# Build timestamped Excel filename
DateForFileName = "FullDatabase+2.5Goals_" + datetime.now().strftime("%d-%m-%Y") + ".xlsx"

# Write multiple tables (Total, Results, Fixtures) to Excel workbook
with pd.ExcelWriter(DateForFileName) as writer:
    dfOverUnderGoalsTotalFullTime_v2_0.to_excel(writer, sheet_name='OverUnderGoalsTotalFullTime', index=False)
    dfresults.to_excel(writer, sheet_name='Results', index=False)
    dffixtures.to_excel(writer, sheet_name='Fixtures', index=False)

# 2. "Treated Data" Dataset 

## 2.1. Filtering Matches with at Least 12 Games Played and a Minimum Low-bound Over-2.5 Probability above 50%
This section refines the ranked dataset to include only matches supported by **sufficient historical data** and **strong scoring potential**.  
By applying thresholds for both the number of games played and conservative Over-2.5-goal probability, it ensures that subsequent analyses are based on **statistically reliable and meaningful observations**.  

---

**This section:**  
- Converts key columns to numeric format to ensure accurate filtering.  
- Retains only teams with **at least 12 games played (GP ≥ 12)** — ensuring data robustness.  
- Keeps matches where the **low-bound Over-2.5-goal probability exceeds 50%**, representing a solid baseline expectation for goal activity.  

In [ ]:
# Start treated dataset generation from the ranked Total table
##########################################
##Treated File
Treated_dfOverUnderGoalsTotalFullTime_v2_0 = dfOverUnderGoalsTotalFullTime_v2_0

# Filter for sufficient sample size (minimum 12 games played)
################################
#Keeping only lines with GP >= 12
Treated_dfOverUnderGoalsTotalFullTime_v2_0.loc[:, 'GP'] = pd.to_numeric(Treated_dfOverUnderGoalsTotalFullTime_v2_0['GP'], errors='coerce')
Treated_dfOverUnderGoalsTotalFullTime_v2_0 = Treated_dfOverUnderGoalsTotalFullTime_v2_0[Treated_dfOverUnderGoalsTotalFullTime_v2_0['GP'] >= 12]

# Filter for strong over-2.5 signal (low-bound home/away probability ≥ 50)
################################
#Keeping only lines with low bound home/away probability >= 50
Treated_dfOverUnderGoalsTotalFullTime_v2_0.loc[:, 'low bound home/away probability'] = pd.to_numeric(Treated_dfOverUnderGoalsTotalFullTime_v2_0['low bound home/away probability'], errors='coerce')
Treated_dfOverUnderGoalsTotalFullTime_v2_0 = Treated_dfOverUnderGoalsTotalFullTime_v2_0[Treated_dfOverUnderGoalsTotalFullTime_v2_0['low bound home/away probability'] >= 50]

## 2.2. Calculating Worst Case and Best Case Scenario Expected Goals
This section introduces **scenario-based projections** for each upcoming match, quantifying both conservative and optimistic goal expectations.  
By comparing each team’s attacking (goals scored) and defensive (goals conceded) averages, it estimates the **lowest and highest plausible total goal outcomes** for every fixture.  

---

**This section:**  
- Computes **worst-case expectations** by taking the *minimum* between a team’s scoring and its opponent’s defensive metrics.  
- Computes **best-case expectations** by taking the *maximum* of the same indicators.  
- Derives **total expected goals** for both best- and worst-case scenarios (sum of home and away expectations).   

The resulting dataset captures both the **floor and ceiling of expected scoring outcomes**, offering a balanced perspective for evaluating potential Over-2.5 goal scenarios.

In [ ]:
# Add visual separators and compute expected goals for worst/best-case match scenarios
Treated_dfOverUnderGoalsTotalFullTime_v2_0[''] = None

# Compute worst-case expected goals for home (lower of GS home or GC away)
Treated_dfOverUnderGoalsTotalFullTime_v2_0['WorstCaseExpGS_Home'] = Treated_dfOverUnderGoalsTotalFullTime_v2_0[['average_GS_Home','average_GC_Away']].min(axis=1)

# Compute worst-case expected goals for away (lower of GS away or GC home)
Treated_dfOverUnderGoalsTotalFullTime_v2_0['WorstCaseExpGS_Away'] = Treated_dfOverUnderGoalsTotalFullTime_v2_0[['average_GS_Away','average_GC_Home']].min(axis=1)

# Compute worst-case total expected goals (sum of both sides’ lower bounds)
Treated_dfOverUnderGoalsTotalFullTime_v2_0['WorstCaseExpResult'] = (
    Treated_dfOverUnderGoalsTotalFullTime_v2_0[['average_GS_Home','average_GC_Away']].min(axis=1) +
    Treated_dfOverUnderGoalsTotalFullTime_v2_0[['average_GS_Away','average_GC_Home']].min(axis=1)
)

# Add spacer column for readability
Treated_dfOverUnderGoalsTotalFullTime_v2_0['       '] = None

# Compute best-case expected goals for home (higher of GS home or GC away)
Treated_dfOverUnderGoalsTotalFullTime_v2_0['BestCaseExpGS_Home'] = Treated_dfOverUnderGoalsTotalFullTime_v2_0[['average_GS_Home','average_GC_Away']].max(axis=1)

# Compute best-case expected goals for away (higher of GS away or GC home)
Treated_dfOverUnderGoalsTotalFullTime_v2_0['BestCaseExpGS_Away'] = Treated_dfOverUnderGoalsTotalFullTime_v2_0[['average_GS_Away','average_GC_Home']].max(axis=1)

# Compute best-case total expected goals (sum of both sides’ upper bounds)
Treated_dfOverUnderGoalsTotalFullTime_v2_0['BestCaseExpResult'] = (
    Treated_dfOverUnderGoalsTotalFullTime_v2_0[['average_GS_Home','average_GC_Away']].max(axis=1) +
    Treated_dfOverUnderGoalsTotalFullTime_v2_0[['average_GS_Away','average_GC_Home']].max(axis=1)
)

# Add another visual separator for final output clarity
Treated_dfOverUnderGoalsTotalFullTime_v2_0['            '] = None

## 2.3.  Evaluating Best- and Worst-Case Expected Goal Outcomes per Fixture
This section assesses each fixture’s **goal-scoring reliability** by comparing simulated best- and worst-case outcomes against the **Over 2.5 goals threshold**.  
By classifying matches according to how both cases perform relative to that benchmark, it highlights fixtures with **consistently strong scoring potential** versus those with limited offensive outlooks.  

---

**This section:**  
- Defines an evaluation function that labels each match as:  
  - **“Both Cases Ok”** – both best and worst scenarios exceed 2.5 goals.  
  - **“Best Case Scenario Ok”** – only the optimistic case surpasses the 2.5 threshold.  
  - **“No Case Ok”** – neither case predicts a likely Over 2.5 outcome.    
- Ranks fixtures based on case strength, **low-bound probability**, and **worst-case expected goals**.  
- Produces a prioritized, interpretable view of matches with the **most statistically supported Over 2.5 prospects**.  

In [ ]:
# Classify each row based on whether worst/best-case totals exceed 2.5
def evaluate_cases(row):
    if row['WorstCaseExpResult'] > 2.5 and row['BestCaseExpResult'] > 2.5:
        return "Both Cases Ok"
    elif row['BestCaseExpResult'] > 2.5:
        return "Best Case Scenario Ok"
    else:
        return "No Case Ok"

# Apply the evaluator to create a CaseEvaluation label per match
Treated_dfOverUnderGoalsTotalFullTime_v2_0['CaseEvaluation'] = Treated_dfOverUnderGoalsTotalFullTime_v2_0.apply(evaluate_cases, axis=1)

# Define desired order for the categorical CaseEvaluation
order = ["Both Cases Ok", "Best Case Scenario Ok", "No Case Ok"]
# Convert CaseEvaluation to an ordered categorical for proper sorting
Treated_dfOverUnderGoalsTotalFullTime_v2_0['CaseEvaluation'] = pd.Categorical(
    Treated_dfOverUnderGoalsTotalFullTime_v2_0['CaseEvaluation'],
    categories=order,
    ordered=True
)
# Sort primarily by case quality, then by low-bound O2.5 (desc), then worst-case total (desc)
Treated_dfOverUnderGoalsTotalFullTime_v2_0 = Treated_dfOverUnderGoalsTotalFullTime_v2_0.sort_values(
    by=["CaseEvaluation", "low bound home/away probability","WorstCaseExpResult"],
    ascending=[True, False,False]
)
# Reset index after sorting for a clean sequential index
Treated_dfOverUnderGoalsTotalFullTime_v2_0.reset_index(drop=True, inplace=True)

## 2.4. View Final "Treated Data" Dataset
Displays the **final treated dataset in a formatted HTML table** for easy visual inspection and scrolling through the results.

In [ ]:
# Render treated dataset as a striped HTML table for visual inspection
HTML(Treated_dfOverUnderGoalsTotalFullTime_v2_0.to_html(classes="table-striped"))

# Render a scrollable HTML view for easier exploration of long tables
HTML(
    Treated_dfOverUnderGoalsTotalFullTime_v2_0.to_html()
    .replace('<table border="1" class="dataframe">', 
             '<table border="1" class="dataframe" style="display:block; height:300px; overflow-y:scroll;">')
)

## 2.5. Downloading the "Treated Data" dataset
This section provides access to the cleaned and processed version of the "Treated Data" dataset.

If you wish to download this dataset, simply run this cell.  

**Note:** Binder does *not* automatically download files.  

To export the dataset:

1. Click the **folder icon** on the left-hand sidebar of the Binder interface.  
2. Locate the generated `.xlsx` file named "TreatedDatabase+2.5Goals_DD-MM-YYYY.xlsx".  
3. Right-click the file and select **“Download.”**

In [ ]:
# Import FileLink for optional download access in Jupyter
from IPython.display import FileLink

# Build timestamped Excel filename for treated dataset
DateForTreatedFileName = "TreatedDatabase_+2.5Goals_" + datetime.now().strftime("%d-%m-%Y") + ".xlsx"

# Write treated dataset to Excel workbook
with pd.ExcelWriter(DateForTreatedFileName) as writer:
    Treated_dfOverUnderGoalsTotalFullTime_v2_0.to_excel(writer, sheet_name='TreatedData', index=False)